## Clasificación de documentos:

# Calculo del género de una película a raíz de su sinopsis

## Parte I: Introducción

Se requiere la implementación de un algoritmo que, previamente entrenado, calcule el __género principal__ de una película en base al conocimiento adquirido.

Los géneros que tendrá en cuenta el algoritmo son: ___acción___, ___comedia___, ___terror___, ___bélico___.

Los __archivos que contienen la sinopsis__ de las películas estarán distribuídos de la siguiente forma: 1) Si forman parte del conjunto de pruebas estarán dentro de la carpeta del conjunto de prueba sin más, que será la carpeta en la que el algoritmo, una vez entrenado, buscará sinopsis para categorizarlas 2) Si forman parte del conjunto de entrenamiento estarán dentro de la carpeta del conjunto de entrenamiento y __a su vez__ dentro de una carpeta que indique su género.

El __formato__ de los archivos será _.txt_, por lo tanto estamos hablando de texto plano.

__Los conocimientos requeridos por parte del usuario__ que ejecutará el algoritmo __son mínimos__: tan sólo necesita __colocar__ los textos, en el formato adecuado, en la carpeta indicada y __ejecutar__ el algoritmo en sí. Únicamente se requiere mayor interacción por parte del usuario si desea cambiar el funcionamiento del algorimo en sí, o ajustar el mismo, como por ejemplo si desea cambiar las palabras clave para cada una de las categorías.

## Parte II: Escaneo de archivos

Empezamos símplemente indicando qué __carpeta__ contiene el __conjunto de entrenamiento__ y el de __prueba__.

In [202]:
ruta_conjunto_entrenamiento = "conjunto_entrenamiento"
ruta_conjunto_prueba = "conjunto_prueba"

#categorías = {"acción", "comedia"}

A continuación, vamos a recorrer nuestra estructura de carpetas para detectar cada uno de los textos a analizar.

Primero __empezamos encontrando__ todos los __archivos ya clasificados__ (conjunto de entrenamiento), según su __categoría__.

In [203]:
import os # Nos ayudaremos de la librería "os" para leer ficheros y carpetas.

# NOTA: Vamos a almacenarlos en conjunto (set) ya que no nos interesa el orden y, además, no permite duplicados (no puede haber dos archivos con el mismo nombre).

total_documentos_entrenamiento = 0 # Util en la parte 4 ("Procesamiento")

archivos_entrenamiento_acción = set()
total_documentos_acción_entrenamiento = 0 # Util en la parte 4 ("Procesamiento")
for file in os.listdir(ruta_conjunto_entrenamiento + "/" + "acción"):
    if file.endswith(".txt"):
        aux = "acción" + "/" + file
        archivos_entrenamiento_acción.add(aux)
        total_documentos_entrenamiento +=1 # Util en la parte 4 ("Procesamiento")
        total_documentos_acción_entrenamiento += 1 # Util en la parte 4 ("Procesamiento")

archivos_entrenamiento_comedia = set()
total_documentos_comedia_entrenamiento = 0 # Util en la parte 4 ("Procesamiento")
for file in os.listdir(ruta_conjunto_entrenamiento + "/" + "comedia"):
    if file.endswith(".txt"):
        aux = "comedia" + "/" + file
        archivos_entrenamiento_comedia.add(aux)
        total_documentos_entrenamiento +=1 # Util en la parte 4 ("Procesamiento")
        total_documentos_comedia_entrenamiento += 1 # Util en la parte 4 ("Procesamiento")

archivos_entrenamiento_terror = set()
total_documentos_terror_entrenamiento = 0 # Util en la parte 4 ("Procesamiento")
for file in os.listdir(ruta_conjunto_entrenamiento + "/" + "terror"):
    if file.endswith(".txt"):
        aux = "terror" + "/" + file
        archivos_entrenamiento_terror.add(aux)
        total_documentos_entrenamiento +=1 # Util en la parte 4 ("Procesamiento")
        total_documentos_terror_entrenamiento += 1 # Util en la parte 4 ("Procesamiento")

archivos_entrenamiento_bélico = set()
total_documentos_bélico_entrenamiento = 0 # Util en la parte 4 ("Procesamiento")
for file in os.listdir(ruta_conjunto_entrenamiento + "/" + "bélico"):
    if file.endswith(".txt"):
        aux = "bélico" + "/" + file
        archivos_entrenamiento_terror.add(aux)
        total_documentos_entrenamiento +=1 # Util en la parte 4 ("Procesamiento")
        total_documentos_bélico_entrenamiento += 1 # Util en la parte 4 ("Procesamiento")

Y, a continuación, procedemos a __encontrar__ los __archivos que querríamos clasificar__ (conjunto de prueba):

In [204]:
# Usamos conjunto por la misma razón de arriba.

archivos_prueba = set()
for file in os.listdir(ruta_conjunto_prueba):
    if file.endswith(".txt"):
        archivos_prueba.add(file)

## Parte III: Elección del conjunto de palabras clave

Para ayudarnos en el estudio de las __palabras clave__ que debemos escoger para cada categoría vamos a realizar un pequeño estudio para determinar las palabras más frecuentes de cada categoría. La elección en sí debemos realizarla a mano puesto que no podemos escoger directamente las más frecuentes puesto que con total seguridad entre las más frecuentes se encontrarán verbos, conectores, preposiciones, pronombres, artículos, nombres propios, etc. que no nos serán de utilidad a la hora de determinar la categoría de una película.

Definimos el siguiente __método que recibirá un conjunto de archivos__ y __contará las palabras__ que aparecen en él __y el número de veces que dichas palabras aparecen__.

In [205]:
def cuenta_palabras_desde_archivos(archivos):
    cuenta_palabras = {}
    
    for archivo in archivos:
        file = open(ruta_conjunto_entrenamiento + "/" + archivo, "r", encoding="latin-1") # Elegimos latin-1 en vez de utf-8 por problemas con las tildes.

        for word in file.read().split(): # Recorremos el fichero, palabra a palabra.
            word = word.lower() # Pasamos la palabra a minúscula.
            # Para perder la menor información posible, reemplazamos ':', ',', ':' y ';', que son los carácteres más típicos que nos podemos encontrar adyacentes a una palabra y que la invalidarían en el siguiente if del algoritmo.
            word = word.replace('.', '')
            word = word.replace(',', '')
            word = word.replace(':', '')
            word = word.replace(';', '')
            if word.isalpha() is True: # Será true cuando todos los caracteres son alfabéticos y hay al menos uno.
                if word in cuenta_palabras:
                    cuenta_palabras[word] += 1 # Si la palabra ya existe, entonces incrementa en 1 el número de veces que hace aparición.
                else:
                    cuenta_palabras[word] = 1 # Si la palabra no existe, la añade (con valor 1 al número de veces que aparece).
    
    return cuenta_palabras

Además, con propósito de limpiar el output que obtendremos vamos a establecer un __umbral__ para desechar todas las palabras que se repitan por debajo del mismo:

In [206]:
umbral_repetición = 4

Y, con el mismo propósito, otro __umbral__ para desechar todas las palabras con una longitud menor a él:

In [207]:
umbral_longitud = 4

Y, por último, un método que use al anterior que, además, nos __ordene las palabras__ (de mayor a menor uso) __y las muestre__:

In [208]:
def cuenta_palabras_desde_archivos_ordenadas(archivos):
    cuenta_palabras = cuenta_palabras_desde_archivos(archivos)
    
    lista = [] # Usamos una lista para poder ordenar las palabras (map).
    for palabra, contador in cuenta_palabras.items():
        lista.append((contador, palabra))

    lista = sorted(lista, reverse = True) # Lo ordenamos y lo invertimos para que las palabras más frecuentes estén arriba.

    for aux in lista:
        if(aux[0] >= umbral_repetición and len(aux[1]) >= umbral_longitud): # Si la palabra supera los umbrales indicados, se muestra.
            print("%s \t %d" % (aux[1], aux[0]))

Palabras más frecuentes en: __ACCIÓN__.

In [209]:
cuenta_palabras_desde_archivos_ordenadas(archivos_entrenamiento_acción)

para 	 75
brian 	 35
pero 	 30
después 	 30
mientras 	 28
policía 	 25
ethan 	 24
bryan 	 24
está 	 23
mcclane 	 21
donde 	 21
quien 	 19
como 	 19
tiene 	 18
equipo 	 17
coche 	 17
agente 	 17
letty 	 16
hunt 	 15
dice 	 15
auto 	 15
ellos 	 14
ella 	 14
cuando 	 14
entonces 	 13
encuentra 	 13
embargo 	 13
novia 	 12
martin 	 12
dominic 	 12
cual 	 12
toretto 	 11
sobre 	 11
luego 	 11
john 	 11
hombres 	 11
también 	 10
llega 	 10
hombre 	 10
escapar 	 10
carrera 	 10
bourne 	 10
ahora 	 10
tras 	 9
sean 	 9
nueva 	 9
logra 	 9
lleva 	 9
grupo 	 9
contra 	 9
antes 	 9
aeropuerto 	 9
vida 	 8
unos 	 8
todo 	 8
película 	 8
owen 	 8
momento 	 8
lugar 	 8
hija 	 8
hasta 	 8
había 	 8
están 	 8
descubre 	 8
virus 	 7
unidos 	 7
todos 	 7
tego 	 7
tarde 	 7
tanto 	 7
sigue 	 7
shaw 	 7
seguridad 	 7
reúne 	 7
plan 	 7
mills 	 7
llegan 	 7
llamada 	 7
lenore 	 7
información 	 7
informa 	 7
gasolina 	 7
gana 	 7
estados 	 7
esposa 	 7
durante 	 7
desde 	 7
comienza 	 7
clay 	 7
ciudad 	 7


Palabras más frecuentes en: __COMEDIA__.

In [210]:
cuenta_palabras_desde_archivos_ordenadas(archivos_entrenamiento_comedia)

tras 	 8
museo 	 8
para 	 6
kahmunrah 	 6
tabla 	 4
siendo 	 4
larry 	 4
jedediah 	 4
figuras 	 4
ellos 	 4
ello 	 4
como 	 4
ayuda 	 4


Palabras más frecuentes en: __TERROR__.

In [211]:
cuenta_palabras_desde_archivos_ordenadas(archivos_entrenamiento_terror)

para 	 51
guerra 	 39
como 	 29
cuando 	 28
está 	 23
ejército 	 22
sargento 	 21
casa 	 21
tras 	 20
soldados 	 19
sobre 	 19
todo 	 18
durante 	 18
lorraine 	 17
carolyn 	 17
aunque 	 17
general 	 16
mientras 	 15
grupo 	 15
donde 	 15
tres 	 13
hombres 	 13
este 	 13
después 	 13
contra 	 13
ante 	 13
hitler 	 12
combate 	 12
tiene 	 11
ella 	 11
april 	 11
todos 	 10
siendo 	 10
quien 	 10
patoso 	 10
misión 	 10
alguien 	 10
unidos 	 9
tiempo 	 9
solo 	 9
pero 	 9
parte 	 9
matar 	 9
lugar 	 9
están 	 9
estados 	 9
ellos 	 9
dice 	 9
descubre 	 9
cómo 	 9
comienza 	 9
teniente 	 8
también 	 8
stauffenberg 	 8
primera 	 8
pelotón 	 8
padre 	 8
mujer 	 8
james 	 8
familia 	 8
entre 	 8
encuentran 	 8
desde 	 8
cual 	 8
capitán 	 8
alemán 	 8
alemanes 	 8
vietnam 	 7
recibe 	 7
raine 	 7
policías 	 7
nuevo 	 7
noche 	 7
llega 	 7
japonés 	 7
isla 	 7
hombre 	 7
historia 	 7
hacia 	 7
habla 	 7
habitación 	 7
exorcismo 	 7
estadounidenses 	 7
estadounidense 	 7
espíritu 	 7
equipo 	 7

Palabras más frecuentes en: __BÉLICO__.

In [212]:
cuenta_palabras_desde_archivos_ordenadas(archivos_entrenamiento_bélico)

Una vez realizado el estudio, inicializamos manualmente las __palabras clave__ de cada __categoría__, usaremos unas 20 palabras para cada una de ellas (podrán repetirse entre categorías).

In [213]:
# Las inicializamos como conjuntos ya que no nos interesa el orden y, además, no se permiten duplicados.
palabras_clave_acción = {"policía", "coche", "agente", "auto", "escapar", "carrera", "seguridad", "llamada", "gasolina", "problemas", "operación", "escapa", "venganza", "prisión", "muerte", "hospital", "carreras", "ataque", "arma", "asesino", "asesina", "asesinos", "terrorista", "terroristas", "persecución", "muerto", "investigación", "destrucción", "bomba"}
palabras_clave_comedia = {}
palabras_clave_terror = {}
palabras_clave_bélico = {"guerra", "ejército", "sargento", "soldados", "general", "grupo", "hombres", "combate", "misión", "unidos", "matar", "teniente", "capitán", "alemán", "alemanes", "vietnam", "estadounidense", "estadounidenses", "soldado", "regimiento", "operación", "oficial", "infantería", "cabo", "enemigo"}

Y, a continuación, añadimos todas las palabras clave de cada categoría a un nuevo conjunto que contenga __todas las palabras clave__.

In [214]:
palabras_clave = set()
# Update nos permite añadir el contenido de un set a otro set
palabras_clave.update(palabras_clave_acción)
palabras_clave.update(palabras_clave_comedia)
palabras_clave.update(palabras_clave_terror)
palabras_clave.update(palabras_clave_bélico)
print("Las palabras clave son: %s" % (palabras_clave))

Las palabras clave son: {'coche', 'problemas', 'persecución', 'venganza', 'sargento', 'combate', 'terroristas', 'enemigo', 'hombres', 'guerra', 'oficial', 'carrera', 'matar', 'misión', 'auto', 'arma', 'llamada', 'escapa', 'alemán', 'capitán', 'asesina', 'operación', 'investigación', 'gasolina', 'unidos', 'muerto', 'general', 'asesinos', 'grupo', 'carreras', 'hospital', 'destrucción', 'asesino', 'muerte', 'estadounidenses', 'prisión', 'teniente', 'estadounidense', 'bomba', 'infantería', 'escapar', 'alemanes', 'terrorista', 'soldado', 'agente', 'seguridad', 'ataque', 'policía', 'ejército', 'cabo', 'regimiento', 'vietnam', 'soldados'}


## Parte IV: Procesamiento

En esta parte se va a llevar a cabo el procesamiento de datos para posteriormente utilizarlos en los algoritmos de __Naive Bayes__ y __kNN__.

Puesto que se deben realizar cálculos distintos para cada algoritmo, dividiremos esta sección en dos subsecciones:

### Procesamiento: Naive Bayes

Para aplicar el algoritmo Naive Bayes primero debemos calcular todos los __P(c)__ ___(probabilidad de "c")___ y los __P(t|c)__ ___(probabilidad de "t" condicionada a "c")___.

En este caso __"c"__ sería nuestra categoría y __"t"__ cada palabra clave.

Primero, vamos a calcular los __P(c)__. Para ello, tenemos que contar el número de documentos de la categoría en cuestión existentes en nuestro conjunto de entrenamiento y dividirlo entre el número total de documentos de nuestro conjunto de entrenamiento. Así pues, por ejemplo, la probabilidad de acción (__P(acción)__) sería el número resultante de dividir el total de documentos catalogados como "acción" de nuestro conjunto de entrenamiento entre el número total de documentos del conjunto de entrenamiento.

Para ahorrarnos repetir los mismos bucles, los cálculos necesarios para elaborar ahora los __P(c)__ ya se han realizado en la parte I del documento, como se indicaba en el código de dicha parte: Allí se calculaban el total de documentos y el total de documentos de cada categoría.

In [215]:
probabilidad_acción = total_documentos_acción_entrenamiento / total_documentos_entrenamiento
print("P(acción) = \t %f" % (probabilidad_acción))

P(acción) = 	 0.530612


In [216]:
probabilidad_comedia = total_documentos_comedia_entrenamiento / total_documentos_entrenamiento
print("P(comedia) = \t %f" % (probabilidad_comedia))

P(comedia) = 	 0.020408


In [217]:
probabilidad_terror = total_documentos_terror_entrenamiento / total_documentos_entrenamiento
print("P(terror) = \t %f" % (probabilidad_terror))

P(terror) = 	 0.020408


In [218]:
probabilidad_bélico = total_documentos_bélico_entrenamiento / total_documentos_entrenamiento
print("P(bélico) = \t %f" % (probabilidad_bélico))

P(bélico) = 	 0.428571


Sólo para asegurarnos, todas las probabilidades deben sumar __~1__ en este apartado:

In [219]:
print("Suma de probabilidades (debe ser ~1) \t %f" % (probabilidad_acción + probabilidad_comedia + probabilidad_terror + probabilidad_bélico))

Suma de probabilidades (debe ser ~1) 	 1.000000


Ahora, para calcular los __P(t|c)__ será un poco más complejo. Para llevar a cabo esta tarea haremos uso de __un diccionario por cada categoría__ que __relacionará palabras clave con su probabilidad condicionada a la categoría del diccionario__. De nuevo, tomaremos la categoría "acción" como ejemplo: el diccionario "probabilidad_palabraclave_acción" recogerá, por cada palabra clave, su probabilidad condiccionada a la categoría acción, es decir, su __P(t|acción__), siendo "t" cada entrada del diccionario.

Pero antes de empezar, definiremos un método, que utilizaremos en los siguientes pasos, que genere cada diccionario deseado como salida y, además, le __aplique un suavizado de LaPlace__:

In [220]:
def crea_diccionario_probabilidades_condicionadas(archivo_entrenamiento_categoría, palabras_clave_categoría):
    probabilidades_condicionadas = {}
    cuenta_palabras = cuenta_palabras_desde_archivos(archivo_entrenamiento_categoría)
    
    número_palabras_clave_totales = len(palabras_clave) # Número de palabras clave que poseemos en total
    número_palabras_clave_categoría = sum(cuenta_palabras.values())
    
    for palabra_clave in palabras_clave:
        if (palabra_clave in palabras_clave_categoría):
            número_veces_aparece_palabra_en_categoría = cuenta_palabras[palabra_clave] # Número de veces que la palabra clave se repite en esta categoría
        else:
            número_veces_aparece_palabra_en_categoría = 0
        result = ((número_veces_aparece_palabra_en_categoría + 1) / (número_palabras_clave_categoría + número_palabras_clave_totales))
        probabilidades_condicionadas[palabra_clave] = result
    
    return probabilidades_condicionadas

También vamos a crear un pequeño método que nos ayude a visualizar las probabilidades condicionadas:

In [221]:
def mostrar_diccionario_probabilidades_condicionadas(diccionario, categoría):
    for entrada in diccionario:
        print("P(%s|%s) = \t %f" % (entrada, categoría, diccionario[entrada]))

Ahora, empezamos con el cálculo en sí:

In [222]:
probabilidad_palabraclave_acción = crea_diccionario_probabilidades_condicionadas(archivos_entrenamiento_acción, palabras_clave_acción)

mostrar_diccionario_probabilidades_condicionadas(probabilidad_palabraclave_acción, "acción")

P(coche|acción) = 	 0.002315
P(problemas|acción) = 	 0.000900
P(persecución|acción) = 	 0.000643
P(venganza|acción) = 	 0.000772
P(sargento|acción) = 	 0.000129
P(combate|acción) = 	 0.000129
P(terroristas|acción) = 	 0.000643
P(enemigo|acción) = 	 0.000129
P(hombres|acción) = 	 0.000129
P(guerra|acción) = 	 0.000129
P(oficial|acción) = 	 0.000129
P(carrera|acción) = 	 0.001415
P(matar|acción) = 	 0.000129
P(misión|acción) = 	 0.000129
P(auto|acción) = 	 0.002058
P(arma|acción) = 	 0.000772
P(llamada|acción) = 	 0.001029
P(escapa|acción) = 	 0.000900
P(alemán|acción) = 	 0.000129
P(capitán|acción) = 	 0.000129
P(asesina|acción) = 	 0.000772
P(operación|acción) = 	 0.000900
P(investigación|acción) = 	 0.000643
P(gasolina|acción) = 	 0.001029
P(unidos|acción) = 	 0.000129
P(muerto|acción) = 	 0.000643
P(general|acción) = 	 0.000129
P(asesinos|acción) = 	 0.000772
P(grupo|acción) = 	 0.000129
P(carreras|acción) = 	 0.000772
P(hospital|acción) = 	 0.000772
P(destrucción|acción) = 	 0.00064

Una vez calculadas todas las probabilidades condicionadas de todas las categorías ya hemos finalizado con este subapartado, pero antes vamos a recordar un par de cosas:

Para acceder a una probabilidad específica símplemente ejecutamos lo siguiente (para el ejemplo obtendremos la __probabilidad de coche__ condicionada a la categoría __acción__, es decir, __P(coche|acción)__):

In [223]:
print("P(coche|acción) = %f" % (probabilidad_palabraclave_acción["coche"]))

P(coche|acción) = 0.002315


Si hubiese una __palabra clave que no pertenece a acción, la probabilidad no sería 0__ ya que estamos usando suavizado. Por ejemplo, podemos comprobarlo con la probabilidad de soldado (que no forma parte de las palabras clave de acción) condicionada a acción (__P(soldado|acción)__):

In [224]:
print("P(soldado|acción) = %f" % (probabilidad_palabraclave_acción["soldado"]))

P(soldado|acción) = 0.000129


### Procesamiento: kNN

# TODO